In [ ]:
#!pip install tweepy
import pandas as pd
import tweepy

# Scraping

In [ ]:
info_api_key = "fSFhW6NJHivhKs88VLtlKQ5t6"
info_api_key_secret = "X7mVc0epNqxlCdRpa4pP2ymRdpbZwnrEr1edW8SG7JhwMgtEnA"
info_bearer_token = "AAAAAAAAAAAAAAAAAAAAAGyulQEAAAAAg7rYt7A8KD2yYFu0datNWtadXOU%3Dou4oAmL2wFiZxrckLd5OCHioEDDRJw4ZEDp7AuuJx7WHIENpGq"

info_client_id = "VzlNTVNlYmw4b21ORUtKdlJiYkc6MTpjaQ"
info_client_secret = "RhKKHZ0bHeq1QqqNsO3Kzw2_dlo0J1strmd9C0n1bziPwn9PiS"

info_acces_token = "1574148627672571904-tcZR37raV54blRyw5Zx0AK99isuFIb"
info_acces_token_secret = "JNyNF4wsrgOuhZ1EDaTXvOJcmZo3GJnXnKRX9K3fPxVcf"

info_callback_uri = "https://wibbo.org"
info_website_url = "https://wibbo.org"


info_scope = ["tweet.read","tweet.write","users.read"]


In [ ]:
def Recuperation_tweets(L_mots,nb):
    client = tweepy.Client(bearer_token=info_bearer_token)

    query = ''
    for m in L_mots:
        query += m + ' OR '
    query=query[:-3]
    query+='lang=en'
    

    #tweets = client.search_recent_tweets(query,max_results=nb,tweet_fields=['context_annotations', 'created_at'], 
                                         #expansions=['author_id'])
    tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
                              tweet_fields=['context_annotations', 'created_at'], expansions=['author_id'],
                              max_results=100).flatten(limit=nb)
    L_tweets=[]
    for tweet in tweets:
        #if len(tweet.context_annotations) > 0: #s'il y a un domaine et l'entité
        L_tweets.append([tweet.data['id'], tweet.data['text'], tweet.data['author_id'], tweet.data['created_at']])
    
    return L_tweets


In [ ]:
L=Recuperation_tweets(["suicide","suicidal","I want to die","help","kill myself",],10000)
df=pd.DataFrame(L,columns=['id','text','auteur','date'])


In [ ]:
#df.to_csv('Tweepy_Suicide_Tweets.csv',sep=';')
#df.to_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Tweepy_Suicide_Tweets.csv',sep=';',index=False)


# Analyse Users

## Histrorique des tweets des personnes à risques

In [ ]:
df_init=pd.read_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Tweepy_Suicide_Tweets_Final.csv',sep=';')
df=df_init.copy()
df.head()

In [ ]:
L_personnes_a_risques=df[df['groupement_all_modele_par_3']==1]['auteur'].to_list()
L_personnes_a_risques=list(set(L_personnes_a_risques))#garder les valeurs uniques
print(len(L_personnes_a_risques))
print(L_personnes_a_risques[:10])

In [ ]:
def Recuperation_tweets_personnes(L_personnes,nb_tweets):
    client = tweepy.Client(bearer_token=info_bearer_token)

    L_tweets=[]
    for p_id in L_personnes:
        tweets = tweepy.Paginator(client.get_users_tweets,id=p_id,tweet_fields=['context_annotations', 'created_at'],
                                  expansions=['author_id'],max_results=100).flatten(limit=nb_tweets)
        
        #return tweets
        
        if tweets != None:
            for tweet in tweets:
            #if len(tweet.context_annotations) > 0: #s'il y a un domaine et l'entité
                L_tweets.append([tweet.data['id'], tweet.data['text'], tweet.data['author_id'], tweet.data['created_at']])
    
    return L_tweets

In [ ]:
L=Recuperation_tweets_personnes(L_personnes_a_risques[:5],1)
L

In [ ]:
len(L_personnes_a_risques)

## Infos des profils

In [ ]:
def Infos_profils(L_users_id):
    client = tweepy.Client(bearer_token=info_bearer_token)
    L_tot=[]
    for i in range(0,len(L_users_id),100):
        a=i
        b=a+100

        users=client.get_users(ids=L_users_id[a:b], user_fields= ['created_at','description','id','location', 
                                                                    'name','protected','public_metrics','username',
                                                                         'verified','verified_type'])
        L_tot.append(users)
                
    L_info_users=[]
    for users in L_tot:
        for user in users.data:
            L_info_users.append([user['id'],user['name'],user['username'],user['created_at'],user['location'],
                                 user['public_metrics']['followers_count'],user['public_metrics']['following_count'],
                                 user['public_metrics']['tweet_count'],user['public_metrics']['listed_count'],
                                 user['protected'],user['verified'],user['verified_type'],user['description']])

            
    df_users=pd.DataFrame(L_info_users,columns=['id','name','username','created_at','location','followers_count',
                                                'following_count','tweet_count','listed_count','protected','verified',
                                                'verified_type','description'])
    
    return df_users
    

In [ ]:
df_users=Infos_profils(L_personnes_a_risques)
df_users.head(10)

In [ ]:
df_users.to_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Infos_users.csv',sep=';',index=False)
                

In [ ]:
df_users=pd.read_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Infos_users.csv',sep=';')
df_users.head(10)              

# Envoyer message

In [54]:
# Authenticate access
auth = tweepy.OAuthHandler(info_api_key, info_api_key_secret)
auth.set_access_token(info_acces_token, info_acces_token_secret)
# Create API handler
api = tweepy.API(auth)

#api
client = tweepy.Client(bearer_token=info_bearer_token)

In [55]:
client = tweepy.Client(bearer_token=info_bearer_token)
screen_name = "axelg99"
twitterid = client.get_user(username=screen_name)
twitterid.data.id

3080430177

In [56]:
message="Si tu as des pensées suicidaires, s'il te plaît contacte immédiatement SOS Amitié pour obtenir de l'aide. Les horaires sont disponibles 24h/24 et 7j/7. Pour plus d'informations, compose le numéro 09 72 39 40 50. Tu n'es pas seul, nous sommes là pour t'aider."
api.send_direct_message(twitterid.data.id, message)


DirectMessage(_api=<tweepy.api.API object at 0x0000023F183F4100>, _json={'type': 'message_create', 'id': '1618919632072572932', 'created_timestamp': '1674815493015', 'message_create': {'target': {'recipient_id': '3080430177'}, 'sender_id': '1574148627672571904', 'message_data': {'text': "Si tu as des pensées suicidaires, s'il te plaît contacte immédiatement SOS Amitié pour obtenir de l'aide. Les horaires sont disponibles 24h/24 et 7j/7. Pour plus d'informations, compose le numéro 09 72 39 40 50. Tu n'es pas seul, nous sommes là pour t'aider.", 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}}}}, type='message_create', id='1618919632072572932', created_timestamp='1674815493015', message_create={'target': {'recipient_id': '3080430177'}, 'sender_id': '1574148627672571904', 'message_data': {'text': "Si tu as des pensées suicidaires, s'il te plaît contacte immédiatement SOS Amitié pour obtenir de l'aide. Les horaires sont disponibles 24h/24 et 7j/7. Pour plus d'i

# Tests

In [ ]:
auth = tweepy.OAuth2BearerHandler(info_bearer_token)
api = tweepy.API(auth)
api

In [ ]:
auth = tweepy.OAuth2AppHandler(info_api_key, info_api_key_secret)
api = tweepy.API(auth)
api

In [ ]:
auth = tweepy.OAuth2UserHandler(info_api_key, info_api_key_secret, info_acces_token, info_acces_token_secret)
api = tweepy.API(auth)
api

In [ ]:
client = tweepy.Client(info_bearer_token)
client

In [ ]:
client.create_tweet(in_reply_to_tweet_id=twitterid[0].id, text='test')

In [ ]:
client = tweepy.Client(info_api_key,info_api_key_secret,info_acces_token,info_acces_token_secret)
client

In [ ]:
oauth2_user_handler = tweepy.OAuth2UserHandler(
    client_id=info_client_id,
    redirect_uri=info_callback_uri,
    scope=info_scope,
    # Client Secret is only necessary if using a confidential client
    client_secret=info_client_secret
)

print(oauth2_user_handler.get_authorization_url())

In [ ]:
auth_url='https://wibbo.org/?state=RVpZhj5neLjMDeqKXJqfJewCi29zUx&code=cjM4UFZIUFdoWlBFS3NSNnRsekpqMjNlQzhvakF2ZmtuR1R2U2dDRFJ6YU01OjE2NzQ4MTQ2NDAzMjg6MToxOmFjOjE'
access_token = oauth2_user_handler.fetch_token(auth_url)

In [ ]:
api = tweepy.API(auth)
api

In [ ]:
client = tweepy.Client(bearer_token=info_bearer_token)

# Create a tweet query
# https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/5-how-to-write-search-queries.md
query = 'suicide lang:en'
tweets = client.search_recent_tweets(query, max_results=100)
for tweet in tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print(tweet.context_annotations)

In [ ]:
client = tweepy.Client(access_token)
client

In [ ]:
twts = api.search(q="Hello World!")

#list of specific strings we want to check for in Tweets
t = ['Hello world!',
    'Hello World!',
    'Hello World!!!',
    'Hello world!!!',
    'Hello, world!',
    'Hello, World!']

for s in twt:
    for i in t:
        if i == s.text:
            sn = s.user.screen_name
            m = "@%s Hello!" % (sn)
            s = api.update_status(m, s.id)

In [ ]:
recipient_id = "1618042845625212928"  # ID of the user
api.send_direct_message(recipient_id, "Hoi hoi")

In [ ]:
info_api_key = "fSFhW6NJHivhKs88VLtlKQ5t6"
info_api_key_secret = "X7mVc0epNqxlCdRpa4pP2ymRdpbZwnrEr1edW8SG7JhwMgtEnA"
info_bearer_token = "AAAAAAAAAAAAAAAAAAAAAGyulQEAAAAAg7rYt7A8KD2yYFu0datNWtadXOU%3Dou4oAmL2wFiZxrckLd5OCHioEDDRJw4ZEDp7AuuJx7WHIENpGq"

info_client_id = "VzlNTVNlYmw4b21ORUtKdlJiYkc6MTpjaQ"
info_client_secret = "RhKKHZ0bHeq1QqqNsO3Kzw2_dlo0J1strmd9C0n1bziPwn9PiS"

info_acces_token = "1574148627672571904-bIjrsPyaC3zLKaYuF9t7rh4PGkPtwT"
info_acces_token_secret = "jknxWvirBnliMgzINTmmOvGZftZnoShpJe4x1seNBdfth"

info_callback_uri = "https://wibbo.org"
info_website_url = "https://wibbo.org"

info_scope = ["tweet.read","tweet.write","users.read"]

oauth2_user_handler = tweepy.OAuth2UserHandler(
    client_id= info_client_id,
    redirect_uri= info_callback_uri,
    scope= info_scope,
    
    # Client Secret is only necessary if using a confidential client
    client_secret= info_client_secret
)

print(oauth2_user_handler.get_authorization_url())
state = input("Tapez l'url : ")
access_token = oauth2_user_handler.fetch_token(state)
client = tweepy.Client(access_token)

# Tweets d'un user

In [ ]:
tweets = client.get_users_tweets("1059331200261767168")
for tweet in tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print(tweet.context_annotations)